In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import *
import matplotlib.pyplot as plt


def gyh(data):
    s = (data - data.min()) / (data.max() - data.min())
    return s


def ycl(path, label):
    data = np.loadtxt(path)

    x_train = []
    x_test = []

    for i in range(10):
        data_new = data[0 + i * 100:102400 + i * 100]
        data_new = data_new.reshape(100, 1024)

        np.random.shuffle(data_new)
        train = gyh(data_new[:70, :])
        test = gyh(data_new[70:, :])
        x_train.append(train)
        x_test.append(test)

    x_train = np.array(x_train).reshape(-1, 1024)
    x_test = np.array(x_test).reshape(-1, 1024)
    y_train = np.array([label for i in range(0, 700)])
    y_test = np.array([label for i in range(0, 300)])

    return x_train, y_train, x_test, y_test


def stackkk(a, b, c, d, e, f, g, h):
    aa = np.vstack((a, e))
    bb = np.hstack((b, f))
    cc = np.vstack((c, g))
    dd = np.hstack((d, h))
    return aa, bb, cc, dd


def dataset():
    '''
    0:正常轴承
    1：内圈受损0.007
    2：滚珠受损0.007
    3：外圈受损0.007
    4: 内圈受损0.014
    5: 滚珠受损0.014
    6: 外圈受损0.014
    7: 内圈受损0.021
    8: 滚珠受损0.021
    9: 外圈受损0.021

    '''
    x_train_1 = []
    y_train_1 = []
    x_test_1 = []
    y_test_1 = []
    x_train0, y_train0, x_test0, y_test0 = ycl('./12K_1792/97.txt', 0)

    x_train1, y_train1, x_test1, y_test1 = ycl('./12K_1792/0.07内圈1792.txt', 1)
    x_train2, y_train2, x_test2, y_test2 = ycl('./12K_1792/0.07球1792.txt', 2)
    x_train3, y_train3, x_test3, y_test3 = ycl('./12K_1792/0.07外圈1792.txt', 3)

    x_train4, y_train4, x_test4, y_test4 = ycl('./12K_1792/0.14内圈1792.txt', 4)
    x_train5, y_train5, x_test5, y_test5 = ycl('./12K_1792/0.14球1792.txt', 5)
    x_train6, y_train6, x_test6, y_test6 = ycl('./12K_1792/0.14外圈1792.txt', 6)

    x_train7, y_train7, x_test7, y_test7 = ycl('./12K_1792/0.21内圈1792.txt', 7)
    x_train8, y_train8, x_test8, y_test8 = ycl('./12K_1792/0.21球1792.txt', 8)
    x_train9, y_train9, x_test9, y_test9 = ycl('./12K_1792/0.21外圈1792.txt', 9)

    x_train_1.extend(
        [x_train0, x_train1, x_train2, x_train3, x_train4, x_train5, x_train6, x_train7, x_train8, x_train9])
    y_train_1.extend(
        [y_train0, y_train1, y_train2, y_train3, y_train4, y_train5, y_train6, y_train7, y_train8, y_train9])
    x_test_1.extend([x_test0, x_test1, x_test2, x_test3, x_test4, x_test5, x_test6, x_test7, x_test8, x_test9])
    y_test_1.extend([y_test0, y_test1, y_test2, y_test3, y_test4, y_test5, y_test6, y_test7, y_test8, y_test9])

    a, b, c, d = x_train_1[0], y_train_1[0], x_test_1[0], y_test_1[0]
    for i in range(1, 10, 1):
        a, b, c, d = stackkk(a, b, c, d, x_train_1[i], y_train_1[i], x_test_1[i], y_test_1[i])

    return a, b, c, d

x_train,y_train,x_test,y_test=dataset()
x_train_=x_train.reshape(-1,1024,1)
x_test_=x_test.reshape(-1,1024,1)
y_train_=utils.to_categorical(y_train,num_classes=10)
y_test_=utils.to_categorical(y_test,num_classes=10)

network = keras.Sequential(
    [
        keras.layers.Input(shape=(1024, 1)),

        keras.layers.Conv1D(filters=64, kernel_size=8, strides=1, padding='SAME'),  #第一个卷积层
        keras.layers.MaxPooling1D(pool_size=2, strides=2),  # 池化层，高宽为2*2，步长为2，相当于将原来的尺寸减半，
        keras.layers.ReLU(),  # 激活函数

        keras.layers.Conv1D(filters=128, kernel_size=8, strides=1, padding='SAME'), #第二个卷积层
        keras.layers.MaxPooling1D(pool_size=2, strides=2),  # 池化层，高宽为2*2，步长为2，相当于将原来的尺寸减半，
        keras.layers.ReLU(),  # 激活函数

        keras.layers.Conv1D(filters=256, kernel_size=8, strides=1, padding='SAME'),  #第三个卷积层
        keras.layers.MaxPooling1D(pool_size=2, strides=2),  # 池化层，高宽为2*2，步长为2，相当于将原来的尺寸减半，
        keras.layers.ReLU(),  # 激活函数

        keras.layers.Flatten(),  # 打平层

        keras.layers.Dense(1024, activation='relu'),  # 全连接层，1024个节点
        keras.layers.Dropout(0.5),
        keras.layers.Dense(512, activation='relu'),  # 全连接层，512个节点
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax'),  # 全连接层，10个节

    ]
)

network.summary()

network.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.categorical_crossentropy,
    metrics=[keras.metrics.categorical_accuracy]
)

history=network.fit(x_train_,y_train_,batch_size=100,epochs=10,validation_data=(x_test_,y_test_))

pred=network.predict(x_test_).argmax(axis=1)
acc=tf.reduce_mean(tf.cast(tf.equal(pred,y_test),tf.float32))
print(acc)

loss=history.history['loss']
acc=history.history[ 'categorical_accuracy']
val_loss=history.history['val_loss']
val_acc=history.history['val_categorical_accuracy']

plt.plot(acc[0:1200])
plt.plot(val_acc[0:1200])
plt.plot(loss[0:1200])
plt.plot(val_loss[0:1200])
plt.ylim(0,1)
plt.legend(['acc','val_acc','loss','val_loss'])
plt.show()

OSError: ./12K_1792/97.txt not found.

In [3]:
from scipy.io import loadmat
import numpy as np
m=loadmat('F:\\python\\12K_1792\\97.mat')
print(m.keys())
x=m['X097_DE_time']
np.set_printoptions(suppress=True)
np.savetxt('F:\\python\\12K_1792\\norm.txt',x)


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\python\\12K_1792\\97.mat'

In [4]:
import math
 
def f(x):
    y = (2**x) + (3*x) - 7
    return y
 
a = float(input("a="))
b = float(input("b="))
ε = float(input("ε="))
 
while math.fabs(a-b) >= ε:
    c = (a+b)/2
 
    if (f(a)*f(c)) < 0:
        b = c
         
    else:
 
        if f(c) == 0:
            a = c
            break
 
        else:
            a = c
 
print("x=",a)


a=1.6
b=2.2
ε=1
x= 1.6


In [5]:
for letter in 'Runoob':
    if letter=='b':
        break
    print('当前字母为：',letter)
var=10
while var>0:
    print('当前变量为：',var)
    var=var-1
    if var==5:
        break
print("Good bye!")

当前字母为： R
当前字母为： u
当前字母为： n
当前字母为： o
当前字母为： o
当前变量为： 10
当前变量为： 9
当前变量为： 8
当前变量为： 7
当前变量为： 6
Good bye!


In [6]:
for letter in 'Runoob':
    if letter=='o':
        pass
        print('执行pass块')
    print('当前字母：',letter)
print("Good bye!")

当前字母： R
当前字母： u
当前字母： n
执行pass块
当前字母： o
执行pass块
当前字母： o
当前字母： b
Good bye!


In [7]:
lc=int(input('输入路程:'))
sj=int(input('输入等待时间:'))
if lc>=10:
    fy=6+(10-3)*1.8+(lc-10)*1.8*1.5
else:
    if lc>3:
        fy=6+(lc-3)*1.8
    else:
        fy=6
    fy=fy+(sj/3)*1
print('车费是:','%10.2f'%fy)

输入路程:20
输入等待时间:30
车费是:      45.60


In [11]:
for i in range(100,1000):
    temp = i
    sum = 0
    while temp:
        sum +=(temp%10) ** 3
        temp //=10
    if sum == i:
        print(i)

153
370
371
407


In [12]:
class MyNumbers:
    def __iter__(self):
        self.a=1
        return self
    def __next__(self):
        x=self.a
        self.a+=1
        return x
myclass=MyNumbers()
myiter=iter(myclass)
print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))

1
2
3
4
5


In [13]:
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self
 
  def __next__(self):
    if self.a <= 20:
      x = self.a
      self.a += 1
      return x
    else:
      raise StopIteration
 
myclass = MyNumbers()
myiter = iter(myclass)
 
for x in myiter:
  print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [14]:
for a in range(1):
       a = int(input('如果是华氏转摄氏，请按1\n,如果是摄氏转华氏，请按2\n'))
if a==1:
        h = float(input("请输入华氏温度"))
        s = (h*1.8)+32
        print("{}华氏度转是{}摄氏度".format(h,s))
else:a==2
s1 = float(input("请输入摄氏温度"))
h1=(s-32)*5/9
print("{}摄氏度转是{}华氏度".format(s1,h1))

如果是华氏转摄氏，请按1
,如果是摄氏转华氏，请按2
1
请输入华氏温度200
200.0华氏度转是392.0摄氏度
请输入摄氏温度100
100.0摄氏度转是200.0华氏度
